In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
import time
from tqdm import tqdm

In [2]:
data_path = '../data/CALC_STRAIGHT_CLEAN/'
files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

In [3]:
print('STRAIGHT SEGMENTS:\t'+ str(len(files)))

STRAIGHT SEGMENTS:	311


In [4]:
X = pd.read_csv('../data/PN chuze/PN ataxie.csv',usecols=[0,1])
X

,File,Ts [s]
0,N-01-1-Char00,0.008
1,A-03-1-Char00,0.008
2,N-04-1-Char00,0.008
3,N-06-1-Char00,0.008
4,N-07-1-Char00,0.008
5,N-740516-Char00,0.017
6,N-535525-Char00,0.017
7,A-736123-Char00,0.017
8,A-715211-Char00,0.017
9,A-766012-Char00,0.017


In [5]:
def mean_xy(df):
    avg = pd.DataFrame()
    for dim in ['x','y']:
        dim_cols = [s for s in df.columns.to_list() if dim in s]
        dim_df = df.filter(dim_cols,axis=1)
        avg[dim] = np.mean(dim_df,axis=1)
    return avg

In [6]:
def angle(df):
    theta = np.zeros((np.size(df,axis=0)))
    dx = np.diff(df.x)
    dy = np.diff(df.y)
    for i in range(len(dx)): 
        theta[i] = math.atan2(dy[i], dx[i]) * 180 / math.pi 
    return theta

In [7]:
stepsR = np.empty(0)
stepsL = np.empty(0)
step_freqR = np.empty(0)
step_freqL = np.empty(0)
t_sampling = np.empty(0)
seg_len = np.empty(0)
t_len = np.empty(0)
starts = np.empty(0)
ends = np.empty(0)
max_dir = np.empty(0)
min_dir = np.empty(0)
mean_dir = np.empty(0)
std_dir = np.empty(0)

for file in tqdm(files):
    subject =  file.split(sep='Char00',maxsplit=1)[0]+'Char00'
    #print(file)
    
    df = pd.read_csv(data_path + file)
    N = np.size(df,axis=0)
    seg_len = np.append(seg_len,N)
    
    T_sampling = X.loc[X['File'] == subject,'Ts [s]'].values
    t_sampling = np.append(t_sampling,T_sampling)
    
    T_len = N*T_sampling
    t_len = np.append(t_len,T_len)
       
    start = df.OrigIndex.iloc[0]
    end = df.OrigIndex.iloc[-1]
    starts = np.append(starts,start)
    ends = np.append(ends,end)
    
    peaksL = np.where(np.diff(df.contactL_clean)>0)[0]
    peaksR = np.where(np.diff(df.contactR_clean)>0)[0]
    #peaksL = np.where(np.diff(df.contactL)>0)[0]
    #peaksR = np.where(np.diff(df.contactR)>0)[0]
    stepsL = np.append(stepsL,len(peaksL))
    stepsR = np.append(stepsR,len(peaksR))
    
    freqL = 1 / (T_len / len(peaksL)) 
    freqR = 1 / (T_len / len(peaksR)) 
    step_freqL = np.append(step_freqL,freqL)
    step_freqR = np.append(step_freqR,freqR)
    
    #DIRECTION CHECK
    # AVERAGE POSITION FROM WHOLE BODY IN XY PLANE
    col_list = df.columns.to_list()
    position_cols = [s for s in col_list if "X" in s]
    xdf = df.filter(position_cols,axis=1)
    
    xy_cols = []
    for col in xdf.columns.to_list():
        if 'x' in col:
            xy_cols.append(xdf.columns.to_list().index(col))
        if 'y' in col:
            xy_cols.append(xdf.columns.to_list().index(col))
    xy = xdf.filter(xdf.columns[xy_cols],axis=1)
    avg = mean_xy(xy)  
    direction = angle(avg)
    
    max_dir = np.append(max_dir,max(savgol_filter(direction,59,2)))
    min_dir = np.append(min_dir,min(savgol_filter(direction,59,2)))
    mean_dir = np.append(mean_dir,np.mean(savgol_filter(direction,59,2)))
    std_dir = np.append(std_dir,np.std(savgol_filter(direction,59,2)))
    
    '''
    if np.std(savgol_filter(direction,59,2)) > 60:
        fig, ax = plt.subplots(4, 1, figsize=(16,5))
        ax[0].plot(avg.x,avg.y)
        ax[0].set_title(file+':AVERAGE FROM ALL POSITIONS: XY projection')
        ax[0].set_ylabel('y')
        ax[0].set_xlabel('x')

        ax[1].plot((np.diff(avg.x)**2 + np.diff(avg.y)**2)**0.5)
        ax[1].plot(savgol_filter((np.diff(avg.x)**2 + np.diff(avg.y)**2)**0.5,59,2),'r')
        ax[1].set_title('DIFFERENCE')
        ax[1].set_ylabel('diff')
        ax[1].set_xlabel('sample')

        ax[2].plot(direction)
        ax[2].plot(savgol_filter(direction,59,2),'r')
        ax[2].set_title('DIRECTION')
        ax[2].set_ylabel('Direction ')
        ax[2].set_xlabel('sample')

        ax[3].plot(np.diff(savgol_filter((np.diff(avg.x)**2 + np.diff(avg.y)**2)**0.5,59,2)),'k')
        ax[3].plot(np.diff(savgol_filter(direction,59,2)),'m')
    '''
    
    

100%|██████████| 311/311 [01:16<00:00,  3.83it/s]


In [8]:
A = pd.DataFrame(data={'File':files, 'IndexStart':starts, 'IndexEnd':ends, 'StepsL':stepsL, 'StepsR':stepsR, 'Ts':t_sampling,
                       'SegLen':seg_len, 'TimeLen':t_len, 'StepFreqL':step_freqL, 'StepFreqR':step_freqR,
                       'MaxDir':max_dir, 'MinDir':min_dir, 'MeanDir':mean_dir,'StdDir':std_dir})
for file in A.File: A.loc[A.File==file,'Type'] = file[0]
A.head()

,File,IndexStart,IndexEnd,StepsL,StepsR,Ts,SegLen,TimeLen,StepFreqL,StepFreqR,MaxDir,MinDir,MeanDir,StdDir,Type
0,N-01-1-Char00part2seg-9_sub_0.csv,58127.0,59961.0,12.0,13.0,0.008,1835.0,14.680,0.817439,0.885559,-65.195771,-134.512166,-80.112084,6.818842,N
1,A-615829-Char00part1seg-4_sub_1.csv,37215.0,37836.0,3.0,2.0,0.008,622.0,4.976,0.602894,0.401929,151.185526,17.011236,90.361148,31.200162,A
2,N-25031996-Char00part3seg-2_sub_0.csv,24585.0,26738.0,14.0,14.0,0.008,2154.0,17.232,0.812442,0.812442,115.613353,-117.315695,-74.752300,12.243346,N
3,A-766012-Char00part1seg-1_sub_5.csv,5114.0,6533.0,11.0,10.0,0.017,1420.0,24.140,0.455675,0.414250,31.103251,-71.406641,-20.846895,20.233188,A
4,N-740516-Char00part3seg-0_sub_0.csv,7316.0,8149.0,13.0,12.0,0.017,834.0,14.178,0.916914,0.846382,-35.739444,-136.718920,-108.459980,16.910178,N


In [9]:
A.to_csv('../data/STRAIGHT_STEP_FREQ_CLEAN.csv',index=False)